# library loading & list defining

In [ ]:
# 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains # 스크롤
from urllib import request # 이미지
import csv
import time
# page로딩과 구조 로딩에 gab이 있어서 원하는 tag나올떄까지 기다리는거
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np # 결측값 삽입용
import re # 정규표현식
import pandas as pd

In [ ]:
# 저장 리스트 정의
main_title_ls = []  # 본제
sub_title_ls = []  # 부제
ncate_ls = []  # 네이버 카테고리
author_ls = []  # 저자
publish_ls = []  # 출판
date_ls = []  # 발행일
tot_page_ls = []  # 전체 쪽수
size_ls = []  # 크기
wei_ls = []  # 무게
kbcate_ls = [] # 교보 카테고리
img_ls = [] # 이미지
book_info_ls = []  # 책소개
book_review_ls = []  # 책서평
index_ls = []
ref_info_ls = [] # 소개 출처 수집용
ref_review_ls = [] # 서평 출처 수집용
ref_index_ls = [] # 목차 출처 수집용

# isbn_title_ls = [] # isbn 교재명
isbn_ls = [] # isbn
set_isbn_ls = [] # set
book_type_ls = [] #제본
ver_ls = []  # 판정보
last_ver_ls = []  # 이전 판 정보
keyword_ls = []  # 키워드
price_ls = []  # 가격
pn_isbn_ls = [] # 페이지 관련 끌기


# function defining

In [ ]:
## 타깃 상품 서치 및 클릭 함수
def find_new():
    try:
#         elements = driver.find_elements(By.CLASS_NAME,f'bookListItem_vendor__d_xBD')
        elements = driver.find_elements(By.PARTIAL_LINK_TEXT,f'판매처')
        print(len(elements),"상품수")
        for i,ele in enumerate(elements):
            print(i, ele.text)
            ele.send_keys(Keys.ENTER)
            driver.implicitly_wait(10)
            print('상품클릭')
            # 네이버 함수
            try:
                naver_info, code = naver() # A
                print('naver 완')
            except:
                code ='X'
            # isbn 함수
#             driver.switch_to.window(driver.window_handles[1]) # 중복
            search_isbn(code, naver_info) # B
    except:
        print('find예외')
        driver.implicitly_wait(10)
        
# 타깃 상품 서치 및 클릭 함수
def target_find(target):
    URL = f'https://search.shopping.naver.com/book/search?bookTabType=ALL&pageIndex=1&pageSize=40&query={target}%20&sort=REL'
    driver.get(url=URL)
    driver.implicitly_wait(15)
    
    try:
#         elements = driver.find_elements(By.CLASS_NAME,f'bookListItem_vendor__d_xBD')
        elements = driver.find_elements(By.PARTIAL_LINK_TEXT,f'판매처')
        print(len(elements),"상품수")
        for i,ele in enumerate(elements):
            print(i, ele.text)
            ele.send_keys(Keys.ENTER)
            driver.implicitly_wait(15)
            print('상품클릭')
            # 네이버 함수
            try:
                naver_info, code = naver() # A
                print('naver 완')
            except:
                code ='X'
            # isbn 함수
#             driver.switch_to.window(driver.window_handles[1]) # 중복
            search_isbn(code, naver_info) # B
    except:
        print('find예외')
        driver.implicitly_wait(10)

In [ ]:
### A
def naver():  # 네이버 도서내에서 실행될 함수
    naver_info = []
    check = 0 
    # 창 전환
    driver.switch_to.window(driver.window_handles[1])
    driver.implicitly_wait(10)
    # 수집 여부 먼저 체크
    ncate, code = sub1_func() # 2
    print(1)
    if code in isbn_ls:
        check += 1
        # return pass # 나가서 작업하는 함수
    else:
        main_title, sub_title = title_func() # 1
    print(2)
    author, publish, date = info_func() # 5
    if date == 'X':
        check += 1
    print(3)
    if check != 0:
        return print('ex')# 나가서 작업하는 함수
    else:
        tot_page, wei, size = sub2_func() # 3

        # 교보문고 링크 존재여부 확인
        book_text = driver.find_elements(By.CLASS_NAME, 'bookSeller_mall_name__S0crJ')
        book_text = [bt.text.split('\n')[0] for bt in book_text]
        if '교보문고' in book_text:
            cate_kb, img = kyobo(code) # 6
            if img == '':
                img = img_func(code)
            else:
                pass
        else:
            cate_kb = ''
            img = img_func(code)
        print(4)
        book_info, book_review, index, ref_info, ref_review, ref_index = new_idx() # 4
        print(5)
    naver_info = [main_title, sub_title, ncate, author, publish, date, tot_page, wei, size, cate_kb,\
                  img, book_info, book_review, index, ref_info, ref_review, ref_index]
    print(6)
    return naver_info, code

# 1
def title_func():
    # 변수 선언
    main_title, sub_title = '',''

    try: # 제목 수집 -부제가 없는지 확인먼저
        sub_title = driver.find_element(By.CLASS_NAME,f'bookTitle_sub_title__B0uMS').text
        main_title = driver.find_element(By.CLASS_NAME,f'bookTitle_book_name__JuBQ2').text

    except:
        main_title = driver.find_element(By.CLASS_NAME,f'bookTitle_title_area__fspvB').text

    return main_title, sub_title

# 2
def sub1_func():  # 네이버 카테고리, isbn번호 수집
    cate_ls = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_inner__YIfRy")
    check = []
    ncate, code = '',''
    for i in range(len(cate_ls)):
        a = cate_ls[i].text.split('\n')
        if a[0] == '카테고리':
            ncate = a[1]

        elif a[0] == 'ISBN':
            code = a[1]
        else:
            pass

    return ncate, code

# 3
def sub2_func():  # 페이지수, 무게, 크기 수집
    sub2 = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_spec__qmQ_N") # 쪽수, 크기(무게)
    # 변수 설정
    tot_page, wei, size = '','',''

    for sb in sub2:
        if '쪽' in sb.text:
            tot_page = sb.text
        elif 'g' in sb.text:
            wei = sb.text
        elif 'm' in sb.text:
            size = sb.text

    return tot_page, wei, size

# 4
def new_idx():
    plus_info = driver.find_elements(By.CLASS_NAME,'infoItem_info_item__1Nm8P')
    # 저장 리스트 정의
    plus_ls = []
    ref = []
    # 변수 설정
    book_info, book_review, index = '','',''
    ref_info, ref_review, ref_index = '','',''

    for pi in plus_info:
        pi_text = pi.text.split('\n')
        if pi_text[0]== '책 소개':
            pi_text = [t for t in pi_text if t != '' ]
            book_info = pi_text[1:-1]
            ref_info = pi_text[-1][:4]
            plus_ls.append('book_info')

        elif pi_text[0]== '출판사 서평':
            pi_text = [t for t in pi_text if t != '' ]
            book_review = pi_text[1:-1]
            ref_review = pi_text[-1][:4]
            plus_ls.append('book_review')

        elif pi_text[0]== '목차':
            pi_text = [t for t in pi_text if t != '' ]
            index = '-+-'.join(pi_text[1:-1]) # 여기 다른방법 고민즁
            ref_index = pi_text[-1][:4]
            plus_ls.append('index')
        else:
            pass

    return book_info, book_review, index, ref_info, ref_review, ref_index

# 5
def info_func():  # 저자, 출판, 발행
    info = driver.find_elements(By.CLASS_NAME, "bookTitle_item_info__IMLyY")
    aut, pub, dat= '','',''

    for i in range(len(info)):
        a = info[i].text.split('\n')
        if a[0] == '저자':
            aut = a[1] # 저자
        elif a[0] == '출판':
            pub = a[1]
        elif a[0] == '발행':
            dat = a[1]
#             if a[1].split('.')[0] not in ['2023','2022','2021','2024']: # 수집 교재의 년도 조절
#                 dat ='X'
#                 print('exit')
#             else:
#                 dat = a[1]
        # else:
        #     pass
    return aut, pub, dat

# 6
def kyobo(code):  # 교보문고 category 수집
    driver.find_element(By.PARTIAL_LINK_TEXT, '교보').click()
    driver.switch_to.window(driver.window_handles[2])
    driver.implicitly_wait(15)
    # 변수설정
    cate_kb, img = '',''
    kb_ls = []
    try:
        kb_txt = driver.find_element(By.CLASS_NAME, 'intro_category_list').text.split('\n')
        for ktt in kb_txt:
            ktt = ktt.split(' > ')
            for kt in ktt:
                if kt not in kb_ls:
                    kb_ls.append(kt)
        cate_kb = '-'.join(kb_ls) # 하나로 묶어서 저장
    except:
        pass

    try:
        img_func_kb(code) # 7
        img = f'{code}.jpg'
    except:
        img = ''

    driver.close()
    driver.implicitly_wait(15)
    driver.switch_to.window(driver.window_handles[1])
    driver.implicitly_wait(15)

    print('정보수집kb')
    return cate_kb, img

# 7
def img_func_kb(code):
    # image
    img_url = driver.find_element(By.XPATH, '//*[@id="contents"]/div[1]/div/div[2]/div/div[2]/div[1]/div[1]/ul/li[1]/div/div[2]/img').get_attribute("src")
    driver.implicitly_wait(15)
    request.urlretrieve(img_url,f"C:/ex_data_1/{code}.jpg") # 저장 주소 지정

# 8 
def img_func(code):
    # image
    img = ''
    img_url = driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[1]/div[1]/div[1]/div[1]/div/img').get_attribute("src")
    driver.implicitly_wait(15)
    request.urlretrieve(img_url,f"C:\\ex_data_1\\{code}.jpg") # 저장 주소 지정
    img = f'{code}.jpg'
    return img

In [ ]:
### B 
def search_isbn(code, naver_info):  # isbn이 단일권인지 세트권인지 판별하는 함수
    if code == 'X':
        print('수집 대상 X')
        pass
    else:
        driver.get(f'https://www.nl.go.kr/seoji/contents/S80100000000.do?schType=simple&schStr={code}')
        driver.implicitly_wait(20)
        num = driver.find_elements(By.CLASS_NAME, 'resultData')
        if len(num) == 1: # 단일 경우 함수
            isbn_info = one_isbn() # 1
            print('단권')
            info_restore(naver_info, isbn_info, code) # 정보 저장 함수
            print('단권 수집 완료')
        elif len(num) > 1:
            x, y = set_code() # 2
            set_code_find(x,y, naver_info, code) # 3
        else:
            pass
            
    driver.close() # 종료
    driver.implicitly_wait(15)
    driver.switch_to.window(driver.window_handles[0])
    print('isbn수집 완')
    
    
# 1
def one_isbn():   # 단일권의 ISBN 수집 코드
    # 변수 설정
    isbn, set_code, book_type, ver = '','','',''
    last_ver, keyword, price, pn_isbn = '','','',''
    print(11)
    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{1}]/div/div/div/a').click()
    driver.implicitly_wait(20)
    # 정보 개수
    num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li')
    for n in range(len(num)):
        a = num[n].text.split('\n')
        if a[0] == 'ISBN':
            isbn = re.sub(r'[^0-9]', '', a[1].split(' ')[0])
        elif a[0] in ['세트ISBN','SET ISBN']:
            set_code = re.sub(r'[^0-9]', '', a[1].split(' ')[0])
        elif a[0] in ['제본형식','제본형식(종이)']:
            book_type = a[-1]
        elif a[0] == '판사항':
            ver = a[-1]
        elif a[0] == '이전판 정보':
            last_ver = re.sub(r'[^0-9]', '', a[1])
        elif a[0] == '형태 및 본문언어':
            pn_isbn = a[1] #세트의 개별권만 수집하도록 변경됨
        elif a[0] == '키워드':
            keyword = a[1]
        elif a[0] in ['가격', '가격정보']:
            if re.sub(r'[^0-9]', '', a[1]) != '':
                price = re.sub(r'[^0-9]', '', a[1])
            else:
                pass
    print(12)
    isbn_info = [isbn, set_code, book_type, ver, last_ver, keyword, price, pn_isbn]
    return isbn_info


# 2
def set_code():  # 세트로 이루어진 정보들중 세트와 개별권 구별 함수
    find_set = driver.find_elements(By.CLASS_NAME, 'dot-list')
    last_idx = []
    first_idx = ''
    for i, fs in enumerate(find_set):
        fs = fs.text.split('\n')
        fs = [f.replace(':', '') for f in fs]
        fs = [f.split(' ')[0] for f in fs]
        print(fs)
        if 'ISBN' not in fs:
            first_idx = i+1
            print(first_idx)
        else:
            last_idx.append(i+1)
    return first_idx,last_idx


# 3 
def set_code_find(x, y, naver_info, code): # 세트권 먼저 수집
    isbn_info, isbn_title = set_code_search(x) # 4
    naver_info[0] = isbn_title
    isbn_info[0] = isbn_info[1] # 세트isbn만 존재인데 이게, 개별과 동일하니깐
    info_restore(naver_info, isbn_info, code) # 정보 저장 함수
    print('세트권 저장 완')
    driver.implicitly_wait(20)
    for i in y:
        isbn_info,isbn_title = set_code_search(i)
        naver_info[0] = isbn_title # 세트권끼리의 제목을 맞추기 위해서 isbn 수집 교재명들로 대체함
        info_restore(naver_info, isbn_info, code)
    print('개별권들 저장완')
    
    
# 4
def set_code_search(x):  # 세트의 개별권 수집
    # 변수 설정
    isbn, set_isbn, book_type, ver = '','','',''
    last_ver, keyword, price, pn_isbn = '','','',''

    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{x}]/div/div/div/a').click()
    driver.implicitly_wait(20)
    num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li')

    isbn_title = driver.find_element(By.XPATH, ' //*[@id="contents"]/div/div[1]/div[1]').text
    isbn_title = ' '.join(isbn_title.split(']')[1:]).strip()
    for n in range(len(num)):
        a = num[n].text.split('\n')
        if a[0] == 'ISBN':
            isbn = re.sub(r'[^0-9]', '', a[1].split(' ')[0])
        elif a[0] in ['세트ISBN','SET ISBN']:
            set_isbn = re.sub(r'[^0-9]', '', a[1].split(' ')[0])
        elif a[0] in ['제본형식','제본형식(종이)'] :
            book_type = a[-1]
        elif a[0] == '판사항':
            ver = a[-1]
        elif a[0] == '이전판 정보':
            last_ver = re.sub(r'[^0-9]', '', a[1])
        elif a[0] == '형태 및 본문언어':
            pni = a[1].split('/')[0]
            pn_isbn = re.sub(r'[^0-9]', '', pni)
        elif a[0] == '키워드':
            keyword = a[-1]
        elif a[0] in ['가격', '가격정보']:
            if re.sub(r'[^0-9]', '', a[1]) != '':
                price = re.sub(r'[^0-9]', '', a[1])
            else:
                pass
    isbn_info = [isbn, set_isbn, book_type, ver, last_ver, keyword, price, pn_isbn]
    driver.back()
    driver.implicitly_wait(20)
    return isbn_info, isbn_title

In [ ]:
# 5
def info_restore(naver_info, isbn_info, code):  # 정보 저장 함수
    # naver 저장
    main_title_ls.append(naver_info[0])
    sub_title_ls.append(naver_info[1])
    ncate_ls.append(naver_info[2])
    author_ls.append(naver_info[3])
    publish_ls.append(naver_info[4])
    date_ls.append(naver_info[5])
    tot_page_ls.append(naver_info[6])
    wei_ls.append(naver_info[7])
    size_ls.append(naver_info[8])
    kbcate_ls.append(naver_info[9])
    img_ls.append(naver_info[10])
    book_info_ls.append(naver_info[11])
    book_review_ls.append(naver_info[12])
    index_ls.append(naver_info[13])
    ref_info_ls.append(naver_info[14])
    ref_review_ls.append(naver_info[15])
    ref_index_ls.append(naver_info[16])
    # isbn 저장
    if isbn_info[0] != '':
        isbn_ls.append(isbn_info[0])
    else:
        isbn_ls.append(code)
    set_isbn_ls.append(isbn_info[1])
    book_type_ls.append(isbn_info[2])
    ver_ls.append(isbn_info[3])
    last_ver_ls.append(isbn_info[4])
    keyword_ls.append(isbn_info[5])
    price_ls.append(isbn_info[6])
    pn_isbn_ls.append(isbn_info[7])

# driver setting

In [1]:
# 드라이버 세팅
ops = webdriver.ChromeOptions()
ops.add_experimental_option('excludeSwitches', ['enable-logging'])
# "C:\Data\chromedriver-win64\chromedriver.exe"
sv = Service(executable_path="C:\Data\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=sv, options=ops)

NameError: name 'webdriver' is not defined

# action
- 이미지 값이 ''인 경우는 수작업으로 가져와야함

In [ ]:
## 1. isbn이 조사되어 있는 경우 
target_ls = # 수집하고자 하는 isbn 리스트

for tg in target_ls:
    target_find(tg)
print('전체 수집 완료')

In [ ]:
book_df = pd.DataFrame({
    'set_isbn':set_isbn_ls,'isbn':isbn_ls,'txtb_nm': main_title_ls, 'txtb_sttl':sub_title_ls,\
    'ncate':ncate_ls, 'txtb_page':tot_page_ls, 'auth_nm':author_ls,'pbsh_nm':publish_ls, \
    'txtb_pblt_dt':date_ls,'txtb_size':size_ls, 'txtb_wght':wei_ls,'img_nm':img_ls,\
    'txtb_itr':book_info_ls, 'txtb_itr_src':ref_info_ls,'txtb_brevw':book_review_ls, \
    'txtb_brevw_src':ref_review_ls,'index':index_ls,'index_src':ref_index_ls,'txtb_price':price_ls,\
    'book_type':book_type_ls,'txtb_edtn_info':ver_ls,'brfe_edtn_isbn':last_ver_ls,'pn_isbn':pn_isbn_ls,\
    'keyword':keyword_ls,'cate_kb':kbcate_ls})

book_df

In [ ]:
book_df.to_excel('저장할 경로 ',encoding = 'utf-8-sig', index=False)

In [ ]:
## 2. 페이지 반복을 통해 직접 수집시
URL = 'https://search.shopping.naver.com/book/search/category?bookTabType=ALL&catId=50005981&pageIndex=1&pageSize=40&sort=DATE'

In [ ]:
driver.get(URL)

action = ActionChains(driver)
action.move_to_element(driver.find_element(By.CLASS_NAME,'Paginator_list_paging__OTDcR')).perform()
# moment = WebDriverWait(driver,15).until(EC.presence_of_element_located((By.CLASS_NAME, 'style_footer__DaTQt')))

i =2
while i<=51:
    print(f'{i-1}페이지')
    action.move_to_element(driver.find_element(By.CLASS_NAME,'Paginator_list_paging__OTDcR')).perform()
    time.sleep(5)
    find_new()
    driver.implicitly_wait(10)
    # moment
    print('for문 페이지 클릭전', i)
    driver.find_element(By.LINK_TEXT,f'{i}').send_keys(Keys.ENTER)
    i += 1
    print('for문 페이지 클릭후')

In [ ]:
book_df = pd.DataFrame({
    'set_isbn':set_isbn_ls,'isbn':isbn_ls,'txtb_nm': main_title_ls, 'txtb_stll':sub_title_ls,\
    'ncate':ncate_ls, 'txtb_page':tot_page_ls, 'auth_nm':author_ls,'pbsh_nm':publish_ls, \
    'txtb_pblt_dt':date_ls,'txtb_size':size_ls, 'txtb_wght':wei_ls,'img_nm':img_ls,\
    'txtb_itr':book_info_ls, 'txtb_itr_src':ref_info_ls,'txtb_brevw':book_review_ls, \
    'txtb_brevw_src':ref_review_ls,'index':index_ls,'index_src':ref_index_ls,'txtb_price':price_ls,\
    'book_type':book_type_ls,'txtb_edtn_info':ver_ls,'brfe_isbn':last_ver_ls,'pn_isbn':pn_isbn_ls,\
    'keyword':keyword_ls,'cate_kb':kbcate_ls})

book_df

In [ ]:
book_df.to_excel('저장할 경로 ',encoding = 'utf-8-sig', index=False)

In [ ]:
# 3. 신간 도서 키워드 검색후 최근 도서 대상
driver.get(urll)
action = ActionChains(driver)

# 특정 요소가 로드될 때까지 기다리기
wait = WebDriverWait(driver, 10)
element_locator = (By.CLASS_NAME, 'Paginator_list_paging__OTDcR')

# 스크롤 내리기 위한 tag
tag = driver.find_element(By.CLASS_NAME, 'Paginator_list_paging__OTDcR')
# 요소가 나타날 때까지 기다림
moment = wait.until(EC.presence_of_element_located(element_locator))

In [ ]:
i = 2 
while True:
    print(f'*** page no.{i-1} ***')
    action.move_to_element(tag).perform()
    time.sleep(5)
    find_new()
    moment
    page_num = driver.find_element(By.CLASS_NAME,'Paginator_list_paging__OTDcR').text
    if str(i) in page_num:
        driver.find_element(By.LINK_TEXT,f'{i}').send_keys(Keys.ENTER)
        i += 1
        print('next page')
    else:
        print('next page is not exist')
        break

In [ ]:
# action = ActionChains(driver)
# action.move_to_element(driver.find_element(By.CLASS_NAME,'Paginator_list_paging__OTDcR')).perform()

In [ ]:
book_df = pd.DataFrame({
    'set_isbn':set_isbn_ls,'isbn':isbn_ls,'txtb_nm': main_title_ls, 'txtb_stll':sub_title_ls,\
    'ncate':ncate_ls, 'txtb_page':tot_page_ls, 'auth_nm':author_ls,'pbsh_nm':publish_ls, \
    'txtb_pblt_dt':date_ls,'txtb_size':size_ls, 'txtb_wght':wei_ls,'img_nm':img_ls,\
    'txtb_itr':book_info_ls, 'txtb_itr_src':ref_info_ls,'txtb_brevw':book_review_ls, \
    'txtb_brevw_src':ref_review_ls,'index':index_ls,'index_src':ref_index_ls,'txtb_price':price_ls,\
    'book_type':book_type_ls,'txtb_edtn_info':ver_ls,'brfe_isbn':last_ver_ls,'pn_isbn':pn_isbn_ls,\
    'keyword':keyword_ls,'cate_kb':kbcate_ls})

book_df

In [ ]:
book_df.to_excel('저장할 경로 ',encoding = 'utf-8-sig', index=False)